In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [44]:
%load_ext autoreload
%autoreload 2
from scripts.text.article_text_processor import ArticleTextProcessor
from scripts.text.basic_text_processor import BasicTextProcessor
from scripts.extractive_summary.ltr.learn_to_rank import LearnToRank

from scripts.conf import TEAMS

from rouge import Rouge

%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Learn to Rank

In [4]:
processor = ArticleTextProcessor()
text_proc = BasicTextProcessor()

In [5]:
all_files = processor.load_json()

In [6]:
season_file = 'premier_league_2019_2020.json'
league_season_teams = TEAMS[season_file.split('.')[0]]

In [64]:
match_dict = all_files[season_file]['https://www.bbc.com/sport/football/49791610']
events = match_dict['events']

## ROUGE

Utilizaremos esta métrica para asignar un score a cada par evento-frase artículo, para generar un target que indique qué evento tiene 
más opciones de aparecer en el resumen. Con este target se entrenará un modelo Learn to rank, de tal forma que se pueda construir un resumen 
con el conjunto de eventos más representativo de cada partido.

In [61]:
ltr = LearnToRank()

Setting target metric to rouge


In [62]:
event_article_list = ltr.create_match_targets(match_dict, verbose=False)

In [138]:
event_article_list

[{'event_ix': 0, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 1, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 2, 'sentence_ix': 13, 'score': 0.037037037037037035},
 {'event_ix': 3, 'sentence_ix': 32, 'score': 0.07142857142857142},
 {'event_ix': 4, 'sentence_ix': 32, 'score': 0.07142857142857142},
 {'event_ix': 5, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 6, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 7, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 8, 'sentence_ix': 23, 'score': 0.05},
 {'event_ix': 9, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 10, 'sentence_ix': 20, 'score': 0.0625},
 {'event_ix': 11, 'sentence_ix': 13, 'score': 0.037037037037037035},
 {'event_ix': 12, 'sentence_ix': 23, 'score': 0.05},
 {'event_ix': 13, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 14, 'sentence_ix': 4, 'score': 0.034482758620689655},
 {'event_ix': 15, 'sentence_ix': 20, 'score': 0.0625},
 {'event_ix': 16, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 17, 'sentence_ix': 32, 'score'

Al usar esta métrica, las correspondencias que se obtienen no tienen demasiado sentido...

In [63]:
ltr.print_scores_info(match_dict, event_article_list)

Score: 0.2926829219750149
Event: Attempt missed. Youri Tielemans (Leicester City) right footed shot from outside the box is too high. Assisted by Wilfred Ndidi with a headed pass.
Nearest article sentence: On a shocking day for Newcastle, Wilfred Ndidi added a late fifth goal with a shot on the turn.

Score: 0.275862064684899
Event: Isaac Hayden (Newcastle United) is shown the red card.
Nearest article sentence: Newcastle boss Steve Bruce on Isaac Hayden's red card: "I've got no complaints, the red card was a poor challenge. 

Score: 0.2758620641141499
Event: Wilfred Ndidi (Leicester City) wins a free kick in the attacking half.
Nearest article sentence: On a shocking day for Newcastle, Wilfred Ndidi added a late fifth goal with a shot on the turn.

Score: 0.2758620641141499
Event: Wilfred Ndidi (Leicester City) wins a free kick in the defensive half.
Nearest article sentence: On a shocking day for Newcastle, Wilfred Ndidi added a late fifth goal with a shot on the turn.

Score: 0.2448

## Distancia coseno

In [65]:
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [66]:
ltr = LearnToRank(target_metric = 'cosine_tfidf')

Setting target metric to cosine_tfidf


In [67]:
event_article_list = ltr.create_match_targets(match_dict, verbose=False, league_season_team=league_season_teams, **count_vec_kwargs)

In [68]:
ltr.print_scores_info(match_dict, event_article_list)

Score: 0.9362460709991904
Event: Substitution, Arsenal. Joe Willock replaces Bukayo Saka.
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.7961554627330942
Event: Foul by Bukayo Saka (Arsenal).
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.7961554627330942
Event: Hand ball by Bukayo Saka (Arsenal).
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.7382013036129462
Event: Bukayo Saka (Arsenal) wins a free kick in the attacking half.
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.7382013036129462
Event: Bukayo Saka (Arsenal) wins a free kick in the attacking half.
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.7045703405951826
Event: Bukayo Saka (Arsenal) wins a free kick on the left wing.
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.5884364770482859
Event: Attempt blocked. Bukayo Saka (Arsenal) right footed shot from the centre of the box is blocked. As

__Examinando tfidf__

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
import pandas as pd

In [69]:
proc_events, proc_article_sents = ltr._process_events_article(match_dict)

In [19]:
len(proc_article_sents)

37

In [20]:
len(proc_events)

125

In [30]:
pipe = Pipeline([('count', CountVectorizer(**count_vec_kwargs)),
                         ('tfid', TfidfTransformer())])

In [31]:
X = pipe.fit_transform(proc_article_sents)

In [187]:
X

<37x521 sparse matrix of type '<class 'numpy.float64'>'
	with 618 stored elements in Compressed Sparse Row format>

In [32]:
tfidf_df = pd.DataFrame(X.todense(), columns=pipe['count'].get_feature_names())

In [13]:
article_sentences = ltr.text_proc.get_sentences(match_dict['article'])
article_sentences_text = [str(sent).replace('\n', '') for sent in article_sentences]

In [35]:
article_sentences_text[1]

'In a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.'

In [36]:
proc_article_sents[1]

'disappointing encounter shadow mighty years gone scott mctominay gave manchester united lead end attritional half rising drive edge'

In [28]:
match_dict['events'][85]

'Foul by Harry Maguire (Manchester United).'

In [29]:
proc_events[85]

'foul harry manchester united'

In [37]:
pd_df_sent = tfidf_df.loc[1]

In [38]:
pd_df_sent[pd_df_sent>0].sort_values(ascending=False)

years gone                 0.177416
lead end                   0.177416
attritional half           0.177416
disappointing              0.177416
disappointing encounter    0.177416
drive edge                 0.177416
encounter                  0.177416
encounter shadow           0.177416
end attritional            0.177416
gave                       0.177416
gave manchester            0.177416
gone                       0.177416
gone scott                 0.177416
half rising                0.177416
lead                       0.177416
attritional                0.177416
shadow                     0.177416
united lead                0.177416
shadow mighty              0.177416
mctominay gave             0.177416
mighty                     0.177416
mighty years               0.177416
rising                     0.177416
rising drive               0.177416
scott                      0.177416
scott mctominay            0.177416
years                      0.159178
drive                      0

In [54]:
events[7]

'Hand ball by Bukayo Saka (Arsenal).'

In [55]:
proc_events[7]

'hand ball bukayo saka arsenal'

In [49]:
X_events = pipe.transform(proc_events)

In [50]:
tfidf_events_df = pd.DataFrame(X_events.todense(), columns=pipe['count'].get_feature_names())

In [52]:
pd_df_sent_event = tfidf_events_df.loc[7]

In [53]:
pd_df_sent_event[pd_df_sent_event>0].sort_values(ascending=False)

ball           0.526185
bukayo saka    0.472097
bukayo         0.472097
saka           0.433720
arsenal        0.298773
Name: 7, dtype: float64

# WMD

WMD (Word Movers Distance) es una distancia basada en la representación de palabras usando Word Embeddings. La principal ventaja de usar word embeddings
es que la distancia puede ser pequeña, aunque no haya palabras en común (sinónimos)

Vamos a probarlo primero en un ejemplo simple.

In [73]:
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [74]:
proc_events[11]

'calum chambers arsenal shown yellow card bad foul'

In [71]:
events

['Lineups are announced and players are warming up.',
 'First Half begins.',
 'Foul by Matteo Guendouzi (Arsenal).',
 'Scott McTominay (Manchester United) wins a free kick in the defensive half.',
 'Granit Xhaka (Arsenal) wins a free kick in the defensive half.',
 'Foul by Jesse Lingard (Manchester United).',
 'Corner,  Arsenal. Conceded by Axel Tuanzebe.',
 'Hand ball by Bukayo Saka (Arsenal).',
 'Corner,  Manchester United. Conceded by Calum Chambers.',
 'Foul by Calum Chambers (Arsenal).',
 'Daniel James (Manchester United) wins a free kick on the left wing.',
 'Calum Chambers (Arsenal) is shown the yellow card for a bad foul.',
 'Offside, Manchester United. Ashley Young tries a through ball, but Harry Maguire is caught offside.',
 'Hand ball by Andreas Pereira (Manchester United).',
 'Offside, Arsenal. Bernd Leno tries a through ball, but Bukayo Saka is caught offside.',
 'Pierre-Emerick Aubameyang (Arsenal) wins a free kick on the left wing.',
 'Foul by Paul Pogba (Manchester Unit